In [1]:
import pandas as pd

covid_data — данные о показателях заболеваемости за каждый день в различных регионах 187 стран. [(источник данных)](https://www.kaggle.com/datasets/sudalairajkumar/novel-corona-virus-2019-dataset)

* date — дата наблюдения;
* province/state — наименование провинции/штата;
* country — наименование страны;
* confirmed — общее число зафиксированных случаев на указанный день;
* deaths — общее число зафиксированных смертей на указанный день;
* recovered — общее число выздоровлений на указанный день.

In [2]:
covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.head())
#display(covid_data.info())

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


country_vaccinations — данные о процессе вакцинирования людей в 221 стране мира. [(источник данных)](https://www.kaggle.com/datasets/gpreda/covid-world-vaccination-progress)

* country — наименование страны;
* date — дата наблюдения;
* total_vaccinations — общее число введённых вакцин в стране на указанный день;
* people_vaccinated — общее число привитых первым компонентом в стране на указанный день;
* people_vaccinated_per_hundred — процент привитых первым компонентом в стране на указанный день (рассчитывается как );
* people_fully_vaccinated — общее число привитых вторым компонентом в стране на указанный день (первый компонент уже был введён им ранее);
* people_fully_vaccinated_per_hundred — процент привитых вторым компонентом в стране на указанный день (рассчитывается как );
* daily_vaccination — ежедневная вакцинация (число вакцинированных в указанный день);
* vaccines — комбинации вакцин, используемые в стране.

In [3]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]
display(vaccinations_data.head())
#display(vaccinations_data.info())

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,Afghanistan,2021-02-22,0.0,0.0,0.0,NaN,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."


Обратите внимание, что признаки confirmed, deaths, recovered, total_vaccination, people_vaccinated, people_fully_vaccinated — это суммарные показатели по стране, то есть с каждым днём они должны расти. Такие признаки называют накопительными.

Возникает большое желание объединить таблицы. Для этого необходимо учитывать следующие нюансы:

1. В таблице covid_data необходимо предварительно рассчитать суммарное 
ежедневное число заболевших во всех провинциях/штатах в каждой стране.

2. В таблицах не совпадает число стран, а иногда и их названия. 

При объединении таблиц по столбцу мы определённо теряем данные 
(в данной задаче потери незначительны). Избежать этого можно ручными
 преобразованиями данных — искать различия в названиях стран в таблицах
  и преобразовывать их. Однако это не является темой данного модуля.

3. Таблицы имеют разные периоды наблюдений (вакцины появились позже, чем сам вирус). 
Объединив данные с типом inner, мы можем потерять большое количество наблюдений в таблице covid_data.

## ПРЕДОБРАБОТКА ДАННЫХ

В таблице __covid_data__:

1. Группируем таблицу по дате и названию страны 
и рассчитываем суммарные показатели по всем регионам. 
Тем самым переходим от данных по регионам к данным по странам:

In [4]:
covid_data = covid_data.groupby(
    ['date', 'country'], 
    as_index=False
)[['confirmed', 'deaths', 'recovered']].sum()

2. Преобразуем даты в формат datetime с помощью функции pd.to_datetime():

In [5]:
covid_data ['date'] = pd.to_datetime(covid_data['date'])

3. Создадим признак больных на данный момент (active). 
Для этого вычтем из общего числа зафиксированных случаев число смертей 
и число выздоровевших пациентов:

In [6]:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

4. Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. 
Для этого отсортируем данные по названиям стран, а затем по датам. 
После этого произведём группировку по странам 
и рассчитаем разницу между «вчера и сегодня» с помощью метода diff():

In [7]:
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

In [8]:
covid_data.head()

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered
11337,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN
11570,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11807,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12051,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12299,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0


В таблице vaccinations_data достаточно будет преобразовать столбцы в формат datetime:

In [9]:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

In [10]:
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86785 entries, 11337 to 17645
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             86785 non-null  datetime64[ns]
 1   country          86785 non-null  object        
 2   confirmed        86785 non-null  float64       
 3   deaths           86785 non-null  float64       
 4   recovered        86785 non-null  float64       
 5   active           86785 non-null  float64       
 6   daily_confirmed  86564 non-null  float64       
 7   daily_deaths     86564 non-null  float64       
 8   daily_recovered  86564 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 6.6+ MB


In [11]:
#За какой период представлены данные в таблице covid_data? 
# В качестве ответа введите даты в формате datetime (без указания времени).
# Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД
covid_data.describe(include=['datetime64']).loc['first':'last']


C:\Users\fga\AppData\Local\Temp\ipykernel_27052\2591061465.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  covid_data.describe(include=['datetime64']).loc['first':'last']


,date
first,2020-01-22 00:00:00
last,2021-05-29 00:00:00


In [12]:
# За какой период представлены данные в таблице vaccinations_data? 
# В качестве ответа введите даты в формате datetime без указания времени.
# Введите ответ в следующем формате (без пробелов): ГГГГ-ММ-ДД-ГГГГ-ММ-ДД
vaccinations_data.describe(include=['datetime64']).loc['first':'last']

C:\Users\fga\AppData\Local\Temp\ipykernel_27052\1041131053.py:4: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  vaccinations_data.describe(include=['datetime64']).loc['first':'last']


,date
first,2020-12-02 00:00:00
last,2021-09-06 00:00:00


In [13]:
display(covid_data.info())
display(vaccinations_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86785 entries, 11337 to 17645
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             86785 non-null  datetime64[ns]
 1   country          86785 non-null  object        
 2   confirmed        86785 non-null  float64       
 3   deaths           86785 non-null  float64       
 4   recovered        86785 non-null  float64       
 5   active           86785 non-null  float64       
 6   daily_confirmed  86564 non-null  float64       
 7   daily_deaths     86564 non-null  float64       
 8   daily_recovered  86564 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 6.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42795 entries, 0 to 42794
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   country                              42795 non-null  object        
 1   date                                 42795 non-null  datetime64[ns]
 2   total_vaccinations                   23457 non-null  float64       
 3   people_vaccinated                    22371 non-null  float64       
 4   people_vaccinated_per_hundred        22371 non-null  float64       
 5   people_fully_vaccinated              19462 non-null  float64       
 6   people_fully_vaccinated_per_hundred  19462 non-null  float64       
 7   daily_vaccinations                   42558 non-null  float64       
 8   vaccines                             42795 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 2.9+ MB


None

In [14]:
# С помощью метода merge() объедините таблицы covid_data и vaccinations_data 
# по столбцам date и country.
#Тип объединения выставьте так, чтобы в результирующую таблицу попали только 
# наблюдения за период, вычисленный в задании 3.1. 
# То есть в результирующую таблицу должны попасть все записи 
# из таблицы covid_data и из её пересечения с vaccinations_data, 
# но не более. 
# Результат объединения занесите в переменную covid_df.
covid_df = covid_data.merge(vaccinations_data,
                            how='left',
                            on=['country','date'])

In [15]:
# С помощью метода merge() объедините таблицы covid_data и vaccinations_data 
# по столбцам date и country.
#Тип объединения выставьте так, чтобы в результирующую таблицу попали только 
# наблюдения за период, вычисленный в задании 3.1. 
# То есть в результирующую таблицу должны попасть все записи 
# из таблицы covid_data и из её пересечения с vaccinations_data, 
# но не более. 
# Результат объединения занесите в переменную covid_df.
covid_df = covid_data.merge(vaccinations_data,
                            how='left',
                            on=['country','date'])
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86785 entries, 0 to 86784
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   date                                 86785 non-null  datetime64[ns]
 1   country                              86785 non-null  object        
 2   confirmed                            86785 non-null  float64       
 3   deaths                               86785 non-null  float64       
 4   recovered                            86785 non-null  float64       
 5   active                               86785 non-null  float64       
 6   daily_confirmed                      86564 non-null  float64       
 7   daily_deaths                         86564 non-null  float64       
 8   daily_recovered                      86564 non-null  float64       
 9   total_vaccinations                   10629 non-null  float64       
 10  people_vac

In [16]:
# Сохраните таблицу covid_df — она понадобится нам в следующем юните.
covid_df.to_csv('data/covid_df.csv',sep=',')

In [17]:
# В получившейся в задании 3.3 таблице covid_df создайте признаки 
# death_rate — общий процент смертей среди зафиксированных случаев (летальность) и
# recover_rate — общий процент случаев выздоровления. 
# Данные характеристики рассчитайте как отношение числа смертей (deaths) 
# и числа выздоровлений (recovered) к числу зафиксированных случаев (confirmed) 
# и умножьте результаты на 100%.
covid_df['death_rate']= covid_df['deaths']/covid_df['confirmed']*100
covid_df['recovered_rate']= covid_df['recovered']/covid_df['confirmed']*100
display(covid_df.iloc[3356])
display(covid_df[covid_df['country']=='United States'])

date                                   2020-09-19 00:00:00
country                                            Armenia
confirmed                                          47154.0
deaths                                               928.0
recovered                                          42551.0
active                                              3675.0
daily_confirmed                                      244.0
daily_deaths                                           2.0
daily_recovered                                      182.0
total_vaccinations                                     NaN
people_vaccinated                                      NaN
people_vaccinated_per_hundred                          NaN
people_fully_vaccinated                                NaN
people_fully_vaccinated_per_hundred                    NaN
daily_vaccinations                                     NaN
vaccines                                               NaN
death_rate                                         1.968

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines,death_rate,recovered_rate
82542,2020-01-22,United States,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0
82543,2020-01-23,United States,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0
82544,2020-01-24,United States,2.0,0.0,0.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0
82545,2020-01-25,United States,2.0,0.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0
82546,2020-01-26,United States,5.0,0.0,0.0,5.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83031,2021-05-25,United States,33166418.0,590941.0,0.0,32575477.0,22756.0,621.0,0.0,287788872.0,164378258.0,48.87,131078608.0,38.97,1750524.0,"Johnson&Johnson, Moderna, Pfizer/BioNTech",1.781745,0.0
83032,2021-05-26,United States,33190470.0,591950.0,0.0,32598520.0,24052.0,1009.0,0.0,289212304.0,165074907.0,49.08,131850089.0,39.20,1703162.0,"Johnson&Johnson, Moderna, Pfizer/BioNTech",1.783494,0.0
83033,2021-05-27,United States,33217995.0,593288.0,0.0,32624707.0,27525.0,1338.0,0.0,290724607.0,165718717.0,49.27,132769894.0,39.48,1618194.0,"Johnson&Johnson, Moderna, Pfizer/BioNTech",1.786044,0.0
83034,2021-05-28,United States,33239963.0,593963.0,0.0,32646000.0,21968.0,675.0,0.0,292099778.0,166388129.0,49.47,133532544.0,39.70,1500632.0,"Johnson&Johnson, Moderna, Pfizer/BioNTech",1.786894,0.0


In [18]:
#Какова максимальная летальность в США (United States) за весь период? 
display(covid_df[(covid_df['country'] == 'United States')]['death_rate'].max())

7.18954248366013

In [19]:
#Чему равен средний процент выздоровевших в России (Russia)?
display(covid_df[(covid_df['country'] == 'Russia')]['recovered_rate'].mean())

67.06273489477655

In [20]:
# Сохраните таблицу covid_df — она понадобится нам в следующем юните.
covid_df.to_csv('data/covid_df.csv',sep=',')